# Frozen Lake: Bellman Equations

In reinforcement learning, an agent is acting in an environment, and learning by trial-and-error to optimize its performance in order to gain maximal cumulative reward. The model of the environment can be formalized by a Markov Decision Process (MDP). To solve the MDP, value functions are defined, which give the expected cumulative reward in a given state. These value functions can be expressed mathematically using the Bellman equations. Solving the MDP comes down to solving these recursive equations. Because of the large number of equations, solving this system is done iteratively applying dynamic programming (DP). In this notebook the Bellman equation applied in the Value Iteration algorithm is discussed and implemented using numpy. Different implementations using loops, list comprehensions, and numpy array operations are tested and compared to see which of these implementations is computationally most efficient.

## Explaining the equation

There are different Bellman equations and the one tested in this notebook is the so called Bellman expectation equation for the state-value function $ V $:

$ V _k(s) = \sum \limits _{a} \left( \pi(a|s) \sum \limits _{s'} P(s,a,s') \left[ R(s,a,s') +\gamma V _{k-1}(s') \right] \right) $

with $ s $ the state, $ a $ the action, $ s' $ the next state, $ \pi $ the policy, $ P $ the state transition probability, $ R $ the reward, $ \gamma $ the discount factor, and $ k $ the iteration number. The equation is solved iteratively, starting with $ V _0(s) = 0 $ for all $ s $. The iterative process stops when a given number of iterations is reached or when a given criterion of convergence is satisfied.

The above equation is applied in the Policy Evaluation algorithm. Both Value Iteration planning and learning algorithms apply the Bellman optimality equation for the state-value function $ V $, which is the same equation as above, except that the first summation $ \sum \limits _{a} $ is replaced by the maximum function $ \max \limits _{a} $. In case of the Value Iteration planning algorithm, there is also no policy $ \pi(a|s) $. Policy Improvement uses a similar equation to determine the action-value function $ Q(s,a) $. In this case the argmax function is used with random tie breaking instead of the summation or max function, and there is no policy $ \pi(a|s) $ either. 

Because all algorithms that determine the state-value function apply a Bellman equation similar to the above equation, testing the latter suffies to optimize all of these algorithms.

## Reorganizing the equation 

The easiest implementation of the iterative process solving the above Bellman equation uses four loops: an iteration loop, a loop for the states $ s $, a loop for the actions $ a $, and a loop for the next states $ s' $. That is the first implementation: loops only. A second implementation replaces the last two loops by a two nested list comprehensions and makes use of Python's built-in sum function.

In numpy, however, it is not computationallly efficient to use loops, and therefore, it is recommended to apply vectorized expressions and array operations as much as possible. For this reason, the tested Bellman equation is reformulated:

$ V _k(s) = \sum \limits _{a} \left( \pi(a|s) \left[ \sum \limits _{s'} P(s,a,s')R(s,a,s') + \sum \limits _{s'} \gamma P(s,a,s') V _{k-1}(s') \right] \right) $


The first implementation using numpy array operations keeps the loop going through all states $ s $, but it replaces the two inner loops for the actions $ a $ and the next states $ s' $: 
- the summation $ \sum \limits _{s'} P(s,a,s')R(s,a,s') $ can be performed using the element-wise matrix multiplication (or Hadamard product) and numpy's sum function for arrays; 
- the summation $ \sum \limits _{s'} \gamma P(s,a,s')V _{k-1}(s') $ can be done applying numpy's dot function for matrix multiplication;
- to execute the outer summation over all actions $ a $, numpy's sum function for arrays can also be used.

The last implementation using numpy array operations even avoids looping through all states $ s $ by reshaping the 3D arrays $ P(s,a,s') $ and $ R(s,a,s') $. Both arrays have size $ n_s $ x $ n_a $ x $ n_s $, with $ n_s $ the number of states and $ n_a $ the number of actions. Reshaping them to matrices of size $ (n_s.n_a) $ x $ n_s $, and performing the inner summutions over all next states $ s' $ for all actions of all states instead of all actions of one state, replaces the outer state $ s $ loop.

These four implemenations are tested in this notebook to verify if they all give the same results. A large number of iterations is performed and time is recorded for each implemenation to see which one is computationally the most efficient.

## Import module ReinforcementLearning 

The stochastic Frozen Lake MDP is used as test case. Therefore the "ReinforcementLearning" module is imported so the required matrices are obtained easily. Also the package "timeit" is imported from which function "time" will be used to record time:

In [1]:
from ReinforcementLearning import *
from timeit import time

A stochastic "FrozenLake" environment is created and its MDP is constructed using class "GymMDP". The uniform random policy is defined using class "UniformRandomPolicy":

In [2]:
env = FrozenLake.make()
mdp = GymMDP(env)
policy = UniformRandomPolicy(env)

Finally some useful variables are assigned:

In [3]:
ns = env.nstates  # number of states
na = env.nactions  # number of actions
niter = 10000  # number of iterations

## Loops only

The first implementation uses three loops to solve the Bellman equation:

In [4]:
time1 = time.time()

Vs = np.zeros(ns)

for i in range(niter):
    for s in range(ns):
        Qsa = []
        for a in range(na):
            qsum = 0.0
            for n in range(ns):
                 qsum += mdp.Psas[s, a, n] * (mdp.Rsas[s, a, n] + mdp.gamma * Vs[n])
            Qsa.append(policy.prob[s, a] * qsum)
        Vs[s] = sum(Qsa)  # replace sum by max for value iteration learning

time2 = time.time()

print("Elapsed time:")
print(time2-time1)
print("\nValue function:")
print(Vs)

Elapsed time:
14.875230073928833

Value function:
[0.0139398  0.01163093 0.02095299 0.01047649 0.01624867 0.
 0.04075154 0.         0.0348062  0.08816993 0.14205316 0.
 0.         0.17582037 0.43929118 0.        ]


## Outer loop and nested list comprehensions 

In the second implementation, the two inner loops are replaced by nested list comprehension and function sum is applied on the resulting list:

In [5]:
time1 = time.time()

Vs = np.zeros(ns)

for i in range(niter):
    for s in range(ns):
        Vs[s] = sum([policy.prob[s, a] *  # replace sum by max for value iteration learning
                     sum([mdp.Psas[s, a, n] *
                          (mdp.Rsas[s, a, n] + mdp.gamma * Vs[n])
                          for n in range(ns)])
                     for a in range(na)])

time2 = time.time()

print("Elapsed time:")
print(time2-time1)
print("\nValue function:")
print(Vs)

Elapsed time:
15.317014455795288

Value function:
[0.0139398  0.01163093 0.02095299 0.01047649 0.01624867 0.
 0.04075154 0.         0.0348062  0.08816993 0.14205316 0.
 0.         0.17582037 0.43929118 0.        ]


## Outer loop and array operations 

The third implementation replaces the two inner loops by numpy array operations:

In [6]:
time1 = time.time()

Vs = np.zeros(ns)
PR = np.sum(mdp.Psas * mdp.Rsas, axis=2)
gPsas = mdp.gamma * mdp.Psas

for i in range(niter):
    for s in range(ns):
        Vs[s] = np.sum(policy.prob[s, :] *  # replace np.sum by np.max for value iteration learning
                       (PR[s, :] + np.squeeze(np.dot(gPsas[s, :, :], Vs))))

time2 = time.time()

print("Elapsed time:")
print(time2-time1)
print("\nValue function:")
print(Vs)

Elapsed time:
1.7662701606750488

Value function:
[0.0139398  0.01163093 0.02095299 0.01047649 0.01624867 0.
 0.04075154 0.         0.0348062  0.08816993 0.14205316 0.
 0.         0.17582037 0.43929118 0.        ]


## Array operations only 

The last implementation replaces the three inner loops by reshaping the numpy arrays:

In [7]:
time1 = time.time()

nsa = ns * na
shape = (ns, na)
prob = np.reshape(policy.prob, (nsa,), order="c")
PR = np.reshape(np.sum(mdp.Psas * mdp.Rsas, axis=2),
                (nsa, ), order="c")
gPsa = mdp.gamma * np.reshape(mdp.Psas, (nsa, ns), order="c")
Vs = np.zeros(ns)

for i in range(niter):
    Qsa = np.reshape(prob * (PR + np.dot(gPsa, Vs)),
                     shape, order="c")
    Vs = np.sum(Qsa, axis=1)  # replace np.sum by np.max for value iteration learning

time2 = time.time()

print("Elapsed time:")
print(time2-time1)
print("\nValue function:")
print(Vs)

Elapsed time:
0.13064908981323242

Value function:
[0.0139398  0.01163093 0.02095299 0.01047649 0.01624867 0.
 0.04075154 0.         0.0348062  0.08816993 0.14205316 0.
 0.         0.17582037 0.43929118 0.        ]


## Conclusion 

It is clear that using numpy array operations is a lot faster than using loops or list comprehensions. Considering the two implementations applying numpy array operations, the implementation that uses no loops at all to calucate the Bellman equation is still significantly more efficient than the implemenation that keeps the outer state $ s $ loop. So it is recommended to make use of this last implementation of the Bellman equations in all planning and learning algorithms. 